In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F 
from pyspark.sql.types import *
from delta import *

In [2]:
builder = SparkSession.builder.appName("olist_demo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")\
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.sql.repl.eagerEval.enabled", "true") \
    .config("spark.executor.memory", "2g")\
    .config("spark.driver.memory", "4g")\
    .config("spark.driver.memory", "4g")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/home/fabio/Repos/olist-pyspark-elt-demo/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/fabio/.ivy2/cache
The jars for the packages stored in: /home/fabio/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3266023e-f017-4c90-860f-fe4ff3baaf6c;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.1 in central
	found io.delta#delta-storage;3.2.1 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 132ms :: artifacts dl 6ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.1 from central in [default]
	io.delta#delta-storage;3.2.1 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   

## Config

In [3]:
environment = '' # dev;uat;prd

In [4]:
path = {
    'brz': {
        'products': f'../../../{environment}/data/bronze/products',
        'product_category_translation': f'../../../{environment}/data/bronze/product_category_translation'
    }, 
    'snk': '../../../data/silver/mtd/mtd_products'
}

## Load

In [5]:
df = {
    'products': spark.read.format('delta').load(path['brz']['products']),
    'product_category_translation': spark.read.format('delta').load(path['brz']['product_category_translation'])
}

In [6]:
df_snk = df['products'].alias('pro').join(
    df['product_category_translation'].alias('pct'),
    on=F.col('pro.product_category_name') == F.col('pct.product_category_name'),
    how='left'
).select(
    F.col('pro.product_id').alias('id_product'),
    F.col('pro.product_category_name').alias('nm_product_category'),
    F.col('pct.product_category_name_english').alias('nm_product_category_english'),
    F.col('pro.product_name_lenght').alias('nm_product_lenght'),
    F.col('pro.product_description_lenght').alias('ds_product_lenght'),
    F.col('pro.product_photos_qty').alias('qt_product_photos'),
    F.col('pro.product_weight_g').alias('qt_product_weight_g'),
    F.col('pro.product_length_cm').alias('qt_product_length_cm'),
    F.col('pro.product_height_cm').alias('qt_product_height_cm'),
    F.col('pro.product_width_cm').alias('qt_product_width_cm')
)

In [7]:
df_snk = df_snk.withColumn("dh_insert_slv", F.current_timestamp())

## Sink

In [8]:
df_snk.write.format('delta')\
.mode(saveMode='overwrite')\
.save(path['snk'])

24/10/03 15:24:40 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
